# conda仮想環境作成
- 環境定義ファイルを格納するディレクトリを作成

In [ ]:
import os

env_dir = "./env"
os.makedirs(env_dir, exist_ok=True)

- 環境定義ファイルを作成

In [ ]:
%%writefile {env_dir}/conda.yml
name: azureml-book-ch5-env
channels:
  - defaults
dependencies:
  - python=3.10.4
  - pip=24.0
  - pip: 
    - azureml-mlflow==1.55.0
    - azure-ai-ml==1.15.0
    - azure-identity==1.16.0
    - python-dotenv==1.0.1
    - ipykernel==6.29.4
    - mlflow==2.12.1
    - lightgbm==4.3.0
    - scikit-learn==1.4.2
    - azureml-fsspec==1.3.1

# モデル開発
- LightGBMで回帰モデルを作成

In [ ]:
import pandas as pd
import numpy as np
import pickle
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import pandas as pd

# RMSEを計算する関数
def rmse(validation, target):
    return np.sqrt(mean_squared_error(validation, target))

# 学習データと検証データの読み込み
df_train = pd.read_csv("../data/Walmert_train.csv")
df_valid = pd.read_csv("../data/Walmert_valid.csv")

# Date列からMonth列とDay列を追加し、Date列を削除
df_train['Month'] = pd.to_datetime(df_train['Date']).dt.month
df_train['Day'] = pd.to_datetime(df_train['Date']).dt.day
df_train = df_train.drop(columns='Date')
df_valid['Month'] = pd.to_datetime(df_valid['Date']).dt.month
df_valid['Day'] = pd.to_datetime(df_valid['Date']).dt.day
df_valid = df_valid.drop(columns='Date')

# ターゲット変数となる列名を指定
col_target = "Weekly_Sales"

# 学習データと検証データを、特徴量とターゲット変数に分割
X_train = df_train.drop(columns=col_target)
y_train = df_train[col_target].to_numpy().ravel()
X_valid = df_valid.drop(columns=col_target)
y_valid = df_valid[col_target].to_numpy().ravel()

# LightGBMのデータセットに変換
train_data = lgb.Dataset(X_train, label=y_train)

# ハイパーパラメータの設定
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05
}

# モデルの学習
model = lgb.train(params, train_data, num_boost_round=100)

# モデルの保存
pickle.dump(model, open("./model.pkl", "wb"))

# 検証データでRMSEを算出
preds = model.predict(X_valid)
print(rmse(df_valid[col_target], preds))


# Azure Machine Learningワークスペース接続
- 接続のためのクレデンシャル取得

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

- Azure Machine Learningワークスペースへ接続するためのハンドルを取得

In [ ]:
ml_client = MLClient(
    credential=credential,
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group_name="<RESOURCE_GROUP>",
    workspace_name="<AML_WORKSPACE_NAME>",
)

# データアセット登録
- 学習データをアセット登録

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# 登録するデータアセットのバージョン指定(例: "1")
VERSION = "1"

# 学習データのパス指定
# local: './<path>/<file>' (対象データは自動的にデフォルトのデータストアへアップロードされる)
# blob:  'wasbs://<container_name>@<account_name>.blob.core.windows.net/<path>/<file>'
# ADLS gen2: 'abfss://<file_system>@<account_name>.dfs.core.windows.net/<path>/<file>'
# Datastore: 'azureml://datastores/<data_store_name>/paths/<path>/<file>'
path = "../data/Walmert_train.csv"

# 学習データのアセット定義
my_data = Data(
    path=path,
    type=AssetTypes.URI_FILE,
    description="ウォルマートの売上履歴 学習データセット",
    name="Walmart_store_sales_train",
    version=VERSION,
)

# 学習データのアセット作成
ml_client.data.create_or_update(my_data)

- 検証データをアセット登録

In [ ]:
# 検証データのパス指定
path = "../data/Walmert_valid.csv"

# 検証データのアセット定義
my_data = Data(
    path=path,
    type=AssetTypes.URI_FILE,
    description="ウォルマートの売上履歴　検証データセット",
    name="Walmart_store_sales_valid",
    version=VERSION,
)

# 検証データのアセット作成
ml_client.data.create_or_update(my_data)

# 環境の作成

In [ ]:
import os
from azure.ai.ml.entities import Environment

custom_env_name = "walmart-store-sales-env"
env_dir = "./env"

custom_job_env = Environment(
    name=custom_env_name,
    description="ウォルマート売上予測モデルの学習ジョブ用の環境",
    tags={"lightgbm": "4.3.0"},
    conda_file=os.path.join(env_dir, "conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
custom_job_env = ml_client.environments.create_or_update(custom_job_env)

print(
    f"{custom_job_env.name} 環境をAzure Machine Learningワークスペースへ登録しました。環境バージョンは {custom_job_env.version} です。"
)

# 学習スクリプト作成
- 学習スクリプト格納先フォルダ作成

In [ ]:
import os

env_dir = "./src"
os.makedirs(env_dir, exist_ok=True)

- 学習スクリプト作成

In [ ]:
%%writefile ./src/main.py
import os
import argparse
import pandas as pd
import mlflow
import mlflow.sklearn
import numpy as np
import lightgbm as lgb
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential


def main():
    """メイン関数"""

    # パラメータ
    parser = argparse.ArgumentParser()
    parser.add_argument("--num_leaves", type=int, default=31, help="学習率")
    parser.add_argument("--learning_rate", type=float, default=0.05, help="1本の木の最大葉枚数")
    parser.add_argument("--registered_model_name", type=str, help="登録するモデル名")
    parser.add_argument("--train_data_path", type=str, help="学習データアセットパス")
    parser.add_argument("--valid_data_path", type=str, help="検証データアセットパス")

    args = parser.parse_args()

    # Azure Machine Learningワークスペースへの接続
    credential = DefaultAzureCredential(exclude_workload_identity_credential=True)
    ml_client = MLClient(
        credential=credential,
		subscription_id="<SUBSCRIPTION_ID>",
		resource_group_name="<RESOURCE_GROUP>",
		workspace_name="<AML_WORKSPACE_NAME>",
    )

    # ロギング開始
    mlflow.start_run()

    # 自動ロギング有効化
    mlflow.lightgbm.autolog()

    ###################
    #<データ準備>
    ###################

    # 学習データと検証データの読み込み
    df_train = pd.read_csv(args.train_data_path)
    df_valid = pd.read_csv(args.valid_data_path)

    # Date列からMonth列とDay列を追加し、Date列を削除
    df_train['Month'] = pd.to_datetime(df_train['Date']).dt.month
    df_train['Day'] = pd.to_datetime(df_train['Date']).dt.day
    df_train = df_train.drop(columns='Date')
    df_valid['Month'] = pd.to_datetime(df_valid['Date']).dt.month
    df_valid['Day'] = pd.to_datetime(df_valid['Date']).dt.day
    df_valid = df_valid.drop(columns='Date')
    
    # ターゲット変数となる列名を指定
    col_target = "Weekly_Sales"

    # 学習データと検証データを、特徴量とターゲット変数に分割
    X_train = df_train.drop(columns=col_target)
    y_train = df_train[col_target].to_numpy().ravel()
    X_valid = df_valid.drop(columns=col_target)
    y_valid = df_valid[col_target].to_numpy().ravel()

    # LightGBMのデータセットに変換
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)

    ####################
    #</データ準備>
    ####################

    ##################
    #<学習>
    ##################
    # ハイパーパラメータの設定
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': args.num_leaves,
        'learning_rate': args.learning_rate
    }

    # モデルの学習
    model = lgb.train(params=params, train_set=train_data,
                        num_boost_round=100, valid_sets=valid_data)


    ###################
    #</学習>
    ###################

    ##########################
    #<モデル登録>
    ##########################
    # 学習済みモデルをAzure Machine Learningワークスペースへ登録
    mlflow.lightgbm.log_model(
        lgb_model=model,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name
    )

    ###########################
    #</モデル登録>
    ###########################

    # ロギング停止
    mlflow.end_run()

if __name__ == "__main__":
    main()

# 学習ジョブ実行
- 学習ジョブの構成を定義

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

train_data_name = ml_client.data.get(name="Walmart_store_sales_train", version="1")
valid_data_name = ml_client.data.get(name="Walmart_store_sales_valid", version="1")

# 学習スクリプト引数設定
inputs = {
    # ジョブの入力として学習データ指定
    "train_data_path": Input(
            type=AssetTypes.URI_FILE,
            path=train_data_name.id
    ),
    # ジョブの入力として検証データ指定
    "valid_data_path": Input(
            type=AssetTypes.URI_FILE,
            path=valid_data_name.id
    ),
    # 1本の木の最大葉枚数
    "num_leaves" : 30,
    # 学習率
    "learning_rate" : 0.04,
    # 登録するモデル名
    "registered_model_name" : "Walmart_store_sales_model"
}

# 学習ジョブの構成
job = command(
    # 学習スクリプト引数
    inputs=inputs,
    # 学習スクリプトの格納場所
    code="./src/",
    # 学習スクリプトの実行コマンド
    command="python main.py --num_leaves ${{inputs.num_leaves}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}} --train_data_path ${{inputs.train_data_path}} --valid_data_path ${{inputs.valid_data_path}}",
    # 環境 ( @latest で最新版を指定。 :バージョン数 でバージョン指定も可)
    environment="walmart-store-sales-env@latest",
    # 実験名
    experiment_name="train_walmart_store_sales_prediction",
    # ジョブの表示名
    display_name="walmart_store_sales_prediction"
)

- 学習ジョブ実行

In [ ]:
job = ml_client.create_or_update(job)
# ジョブ実行が完了するまで待機
ml_client.jobs.stream(job.name)